#  Home Work 3
## Linear Regression: Overfitting and Regularization

In this task, we will see, by examples, how the linear models are overfitting, we will understand why this happens, and we will find out how to diagnose and control overfitting.

In all cells where the comment with instructions was written, you need to write the code that executes these instructions. The rest of the cells with the code (without comments) just need to be executed. In addition, the assignment requires answering questions; The answers must be entered after the selected word "Answer:".

We remind you that you can use the combination Shift + Tab to see the help of any method or function (find out what arguments it has and what it does). Clicking Tab after the object name and point allows you to see what methods and variables this object has.


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

We will work with the __"bikes_rent.csv"__ dateset, in which calendar information and weather conditions characterizing the automated bicycle rental points are recorded daily, as well as the number of rentals that day. The latter will be predicted; Thus, we will solve the problem of regression. https://www.kaggle.com/c/bike-sharing-demand

### Getting to know the data

Load the dataset using the __pandas.read_csv__ function into the __df__ variable. Output the first 5 lines to make sure that the data is correctly read:

In [ ]:
# Read the data and output the first 5 lines


For each rental day, the following characteristics are known :

***season*** :  1 = spring, 2 = summer, 3 = fall, 4 = winter 

***yr*** : 0 - 2011, 1 - 2012

***mnth*** : from 1 to 12

***holiday*** : whether the day is considered a holiday

***weekday*** : from 0 to 6

***workingday*** : whether the day is neither a weekend nor holiday

***weathersit*** : 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
    
***temp*** : temperature in Celsius

***atemp*** : "feels like" temperature in Celsius

***hum*** : relative humidity

***windspeed(mph)*** :   wind speed mph

***windspeed(ms)*** :   wind speed ms

***cnt*** : number of total rentals


So, we have real, binary and nominal (ordinal) signs, and with all of them you can work with both material. With nominal signs, you can also work as a material, because they are given an order. Let's look at the charts, how the target attribute depends on the rest

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10))
for idx, feature in enumerate(df.columns[:-1]):
    df.plot(feature, "cnt", subplots=True, kind="scatter", ax=axes[idx / 4, idx % 4])

__ Block 1. Answer the questions: __
1. What is the nature of the dependence of the number of rent a month?
    * answer:
1. Specify one or two characteristics, from which the number of rentals is most likely linearly dependent
    * answer:

Let's more strictly estimate the level of linear dependence between the characteristics and the target variable. A good measure of the linear relationship between the two vectors is the Pearson correlation. In pandas, it can be calculated using two methods of a dataframe: corr and corrwith. The df.corr method calculates the correlation matrix of all characteristics from the dataframe. The df.corrwith method needs to submit one more dataframe as an argument, and then it will calculate pairwise correlations between the characteristics of df and this dataframe.

In [ ]:
# Compute the correlation of all attributes except the last, with the latter using the corrwith method:


In the sample, there are signs that correlate with the target, and therefore the problem can be solved by linear methods.

The graphs show that some of the signs are similar to each other. Therefore, let us also calculate the correlation between the numerical examples.

In [ ]:
# Calculate the pairwise correlations between the temp, atemp, hum, windspeed (mph), windspeed (ms), and cnt
# using the corr method:


On the diagonals, as it should be, there are only one. However, there are two more pairs of strongly correlated columns in the matrix: temp and atemp (correlate by nature) and two windspeed columns (because this is simply the translation of some units into others). Further we will see that this fact negatively affects the learning of the linear model.

Finally, let's look at the average characteristics (the mean method) in order to estimate the scale of features and the proportion of 1 for binary features.

In [2]:
# Output average characteristics


The samples have a different scale, which means that for further work we better normalize the matrix of the objects-features.

### Problem one: collinear features

So, in our data, one feature duplicates the other, and there are two very similar ones. Of course, we could remove duplicates at once, but let's see how the model would have been trained if we had not noticed this problem.

First, we scale, or standardize the characteristics: from each feature, subtract its average and divide by the standard deviation. This can be done with the scale method.

In addition, you need to shufle the sample, this is required for cross-validation.

In [ ]:
from sklearn.preprocessing import scale
from sklearn.utils import shuffle

In [ ]:
df_shuffled = shuffle(df, random_state=123)
X = scale(df_shuffled[df_shuffled.columns[:-1]])
y = df_shuffled["cnt"]

Let's train the linear regression on our data and look at the weights of the features.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Create a linear regressor object, train it on all data and output the weights of the model
# (weights are stored in the coef_ variable of the regressor class).



In [ ]:
# You can output pairs (attribute name, weight) using the zip function built into the python language
# The characteristic names are stored in the variable df.columns
#temp=pd.DataFrame(list(zip(df.columns,np.around(lin_reg.coef_, decimals=3))))
#temp.index.name='№'
#temp.columns=['features', 'weights']

We see that the weights for linearly dependent characteristics are significantly larger in modulus than in other characters.

To understand why this happened, let us recall the analytical formula by which the weights of the linear model in the method of least squares are calculated:

$ w = (X ^ TX) ^ {- 1} X ^ T y $.

If in X there are collinear (linearly dependent) columns, the matrix $ X ^ TX $ becomes degenerate, and the formula ceases to be correct. The more dependent the characteristics, the smaller the determinant of this matrix and the worse the approximation of $ Xw \approx y $. Such a situation is called the problem of _multicollinearity_

With a pair of temp-atemp slightly less correlating variables, this did not happen, but in practice it is always worthwhile to closely monitor the coefficients at similar characteristics.

__The solution__ of the multicollinearity problem consists in the _regularization_ of the linear model. To the optimized functional add L1 or L2 the norm of the weights multiplied by the regularization coefficient $\alpha$. In the first case, the method is called Lasso, and in the second, Ridge.

Teach the Ridge and Lasso regressors with default parameters and make sure the problem with the scales is resolved.

In [ ]:
from sklearn.linear_model import Lasso, Ridge

In [4]:
# Train a linear model with L1-regularization and output weights


In [3]:
# Train a linear model with L2-regularization and output weights


### The second problem: non-informative features

In contrast to L2-regularization, L1 nullifies weights at some traits. 

Let's observe how the weights change with increasing regularization factor $\alpha$ 

In [ ]:
alphas = np.arange(1, 500, 50)
coefs_lasso = np.zeros((alphas.shape[0], X.shape[1])) # matrix of weights of size (number of regressors) x (number of features)
coefs_ridge = np.zeros((alphas.shape[0], X.shape[1]))
# For each coefficient value from alphas, train the Lasso regressor
# and write down the weights in the corresponding row of the coefs_lasso matrix (remember the built-in python function enumerate)
# and then train the Ridge and write down the weights in coefs_ridge.

We visualize the dynamics of weights with increasing regularization parameter:

In [ ]:
plt.figure(figsize=(8, 5))
for coef, feature in zip(coefs_lasso.T, df.columns):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3))
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Lasso")

plt.figure(figsize=(8, 5))
for coef, feature in zip(coefs_ridge.T, df.columns):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3))
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Ridge")

The answers to the following questions can be given by looking at the graphs or by outputting the coefficients for printing.

__ Block 2. Answer the questions  __:
1. Which regularizer (Ridge or Lasso) aggressively reduces weight for the same alpha?
    * Answer:
1. What happens to the Lasso weights, if alpha is made very large? Explain why this happens.
    * Answer:
1. Is it possible to assert that Lasso excludes one of the feature of windspeed for any value of alpha> 0? And Ridge? It is assumed that the regularizer excludes the attribute if the coefficient of it is <1e-3.
    * Answer:
1. Which of the regularizers is suitable for selecting non-informative features?
    * Answer:


Further we will work with Lasso.

So, we see that when the alpha is changed, the model selects the coefficients of the features in different ways. We need to choose the best alpha.

For this, first, we need a quality metric. We will use the optimized functional of the method of least squares, that is, Mean Square Error, as the metric.

Secondly, it is necessary to understand what data this metric is to be counted on. You can not select alpha by the MSE value on the training sample, because then we will not be able to estimate how the model will make predictions on the new data for it. If we select one splitting of the sample into a training and test (called holdout), then we will adjust to specific "new" data, and again we can retrain. Therefore, we will make several partitions of the sample, at each try different values ​​of alpha, and then average the MSE. It is most convenient to make such cross-sections by cross-validation, that is, to divide the sample into K parts or blocks, and each time to take one of them as a test, and from the remaining blocks make up a training sample.

To do cross-validation for regression in sklearn is quite simple: for this there is a special regressor, __LassoCV__, which takes an alpha list on the input and calculates MSE for cross-validation for each of them. After the training (if you leave the parameter cv = 3 by default), the regressor will contain the variable __mse\_path\___, the matrix of the size len (alpha) x k, k = 3 (the number of blocks in the cross-validation) containing the MSE values on the test for the corresponding starts . In addition, in the variable alpha\_ the selected value of the regularization parameter will be stored, and in coef\_, traditionally, the weights corresponding to this alpha_ will be stored.

Note that the regressor can change the order in which it passes through alphas; It is better to use the variable regressor alphas_ to match the MSE matrix.

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
# Traine the LassoCV regressor on all regularization parameters from alpha
# Construct a graph of _ averaged_ over MSE lines depending on alpha 
# Output the selected alpha, as well as the "feature-coefficient" pairs for the trained coefficient vector
alphas = np.arange(1, 100, 5)


So, we have chosen some regularization parameter. Let's see what we would choose alpha if we only split the sample once into the training and test, that is, consider the MSE trajectories corresponding to the individual sampling blocks.

In [ ]:
# Output the alpha values corresponding to the MSE minima on each partition (that is, by the columns).
# On three separate graphs, visualize the columns .mse_path_


On each partition, the optimal value of alpha is its own, and it corresponds to a large MSE on other partitions. It turns out that we are tuning to specific training and control samples. When choosing alpha for cross-validation, we choose something "average", which will give an acceptable value for the metric on different partitions of the sample.

Finally, as is the case in data analysis, let's interpret the result.

__ Block 3. Answer the questions: __
1. In the last trained model, select the 4 features with the largest (positive) coefficients (and write them out), look at the visualization of cnt dependencies on these attributes, which we drew in the "Introduction to Data" block. Is the increasing linear dependence of cnt on these characteristics from the graphs visible? Is it logical to argue (out of common sense) that the greater the significance of these signs, the more people will want to take bicycles?
    * Answer:
1. Select 3 features with the largest negative coefficients (and write them out), look at the corresponding visualizations. Is a decreasing linear dependence visible? Is it logical to argue that the greater the magnitude of these signs, the less people will want to take bicycles?
    * Answer:
1. Write down the features with coefficients close to zero (<1e-3). Why do you think the model excluded them from the model (again look at the charts)? Is it true that they do not in any way affect the demand for bicycles?
    * Answer:

### Conclusion
So, we looked at how you can monitor the adequacy of the linear model, how to select the attributes and how to correctly, if possible, not adjusting to any particular portion of the data, to select the regularization coefficient.

It is worth noting that using cross-validation it is convenient to select only a small number of parameters (1, 2, maximum 3), because for each allowed combination of them we have to train the model several times, and this is a time-consuming process, especially if it is necessary to be trained on large volumes data.